# 1 网络架构

### SSD网络架构（简易版）

<img src='img(wr)\1_1.png'>

### SSD网络架构（细节版）

<img src='img(wr)\1_2.png'>

### SSD在vgg-16网络架构的基础上，将vgg-16后边两层的全连接层（fc6,fc7）变换为了卷积层，并在conv7之后添加了用于提取特征的额外层。

## 前半部分：在vgg-16基础上修改

In [1]:
def vgg(cfg, i, batch_norm=False):
    layers = []
    in_channels = i
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        elif v == 'C':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2, ceil_mode=True)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    pool5 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
    
    #如下，可以看到原有的全连接层fc6,fc7被改为了conv6，conv7
    conv6 = nn.Conv2d(512, 1024, kernel_size=3, padding=6, dilation=6)
    conv7 = nn.Conv2d(1024, 1024, kernel_size=1)
    layers += [pool5, conv6,
               nn.ReLU(inplace=True), conv7, nn.ReLU(inplace=True)]
    return layers

## 后半部分：在vgg后额外添加卷积层，用于提取不同尺度的特征

In [2]:
def add_extras(cfg, i, batch_norm=False):
    # Extra layers added to VGG for feature scaling
    layers = []
    in_channels = i
    flag = False
    for k, v in enumerate(cfg):
        if in_channels != 'S':
            if v == 'S':
                layers += [nn.Conv2d(in_channels, cfg[k + 1],
                           kernel_size=(1, 3)[flag], stride=2, padding=1)]
            else:
                layers += [nn.Conv2d(in_channels, v, kernel_size=(1, 3)[flag])]
            flag = not flag
        in_channels = v
    return layers

## 以下为另外一种构建网络的方法，更加直观

In [3]:
def __init__(self):
    super(SSD,self).__init__()
    self.vgg = []

    #vgg-16模型
    self.vgg.append(nn.Conv2d(in_channels=3,out_channels=64,kernel_size=3,stride=1,padding=1))#conv1_1
    self.vgg.append(nn.ReLU(inplace=True))
    self.vgg.append(nn.Conv2d(in_channels=64,out_channels=64,kernel_size=3,stride=1,padding=1))#conv1_2
    self.vgg.append(nn.ReLU(inplace=True))
    self.vgg.append(nn.MaxPool2d(kernel_size=2,stride=2))#maxpool1
    self.vgg.append(nn.Conv2d(in_channels=64,out_channels=128,kernel_size=3,stride=1,padding=1))#conv2_1
    self.vgg.append(nn.ReLU(inplace=True))
    self.vgg.append(nn.Conv2d(in_channels=128,out_channels=128,kernel_size=3,stride=1,padding=1))#conv2_2
    self.vgg.append(nn.ReLU(inplace=True))
    self.vgg.append(nn.MaxPool2d(kernel_size=2,stride=2))#maxpool2
    self.vgg.append(nn.Conv2d(in_channels=128,out_channels=256,kernel_size=3,stride=1,padding=1))#conv3_1
    self.vgg.append(nn.ReLU(inplace=True))
    self.vgg.append(nn.Conv2d(in_channels=256,out_channels=256,kernel_size=3,stride=1,padding=1))#conv3_2
    self.vgg.append(nn.ReLU(inplace=True))
    self.vgg.append(nn.Conv2d(in_channels=256,out_channels=256,kernel_size=3,stride=1,padding=1))#conv3_3
    self.vgg.append(nn.ReLU(inplace=True))
    self.vgg.append(nn.MaxPool2d(kernel_size=2,stride=2,ceil_mode=True))#maxpool3
    self.vgg.append(nn.Conv2d(in_channels=256,out_channels=512,kernel_size=3,stride=1,padding=1))#conv4_1
    self.vgg.append(nn.ReLU(inplace=True))
    self.vgg.append(nn.Conv2d(in_channels=512,out_channels=512,kernel_size=3,stride=1,padding=1))#conv4_2
    self.vgg.append(nn.ReLU(inplace=True))
    self.vgg.append(nn.Conv2d(in_channels=512,out_channels=512,kernel_size=3,stride=1,padding=1))#conv4_3
    self.vgg.append(nn.ReLU(inplace=True))
    self.vgg.append(nn.MaxPool2d(kernel_size=2,stride=2))#maxpool4
    self.vgg.append(nn.Conv2d(in_channels=512,out_channels=512,kernel_size=3,stride=1,padding=1))#conv5_1
    self.vgg.append(nn.ReLU(inplace=True))
    self.vgg.append(nn.Conv2d(in_channels=512,out_channels=512,kernel_size=3,stride=1,padding=1))#conv5_2
    self.vgg.append(nn.ReLU(inplace=True))
    self.vgg.append(nn.Conv2d(in_channels=512,out_channels=512,kernel_size=3,stride=1,padding=1))#conv5_3
    self.vgg.append(nn.ReLU(inplace=True))
    self.vgg.append(nn.MaxPool2d(kernel_size=3,stride=1,padding=1))#maxpool5
    self.vgg.append(nn.Conv2d(in_channels=512,out_channels=1024,kernel_size=3,padding=6,dilation=6))#conv6
    self.vgg.append(nn.ReLU(inplace=True))
    self.vgg.append(nn.Conv2d(in_channels=1024,out_channels=1024,kernel_size=1))#conv7
    self.vgg.append(nn.ReLU(inplace=True))
    self.vgg = nn.ModuleList(self.vgg)


    #以下为额外层
    self.conv8_1 = nn.Sequential(
        nn.Conv2d(in_channels=1024,out_channels=256,kernel_size=1),
        nn.ReLU(inplace=True)
    )
    self.conv8_2 = nn.Sequential(
        nn.Conv2d(in_channels=256,out_channels=512,kernel_size=3,stride=2,padding=1),
        nn.ReLU(inplace=True)
    )
    self.conv9_1 = nn.Sequential(
        nn.Conv2d(in_channels=512,out_channels=128,kernel_size=1),
        nn.ReLU(inplace=True)
    )
    self.conv9_2 = nn.Sequential(
        nn.Conv2d(in_channels=128,out_channels=256,kernel_size=3,stride=2,padding=1),
        nn.ReLU(inplace=True)
    )
    self.conv10_1 = nn.Sequential(
        nn.Conv2d(in_channels=256,out_channels=128,kernel_size=1),
        nn.ReLU(inplace=True)
    )
    self.conv10_2 = nn.Sequential(
        nn.Conv2d(in_channels=128,out_channels=256,kernel_size=3,stride=1),
        nn.ReLU(inplace=True)
    )
    self.conv11_1 = nn.Sequential(
        nn.Conv2d(in_channels=256, out_channels=128, kernel_size=1),
        nn.ReLU(inplace=True)
    )
    self.conv11_2 = nn.Sequential(
        nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1),
        nn.ReLU(inplace=True)
    )

# 2 训练模型

### 导入需要的包

In [4]:
from data import *
from utils.augmentations import SSDAugmentation
from layers.modules import MultiBoxLoss
from ssd import build_ssd
import os
import sys
import time
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.nn.init as init
import torch.utils.data as data
import numpy as np
import argparse

### 设置超参数

In [5]:
def str2bool(v):
    return v.lower() in ("yes", "true", "t", "1")

parser = argparse.ArgumentParser(
    description='Single Shot MultiBox Detector Training With Pytorch')
train_set = parser.add_mutually_exclusive_group()
parser.add_argument('--dataset', default='VOC', choices=['VOC', 'COCO'],     #选择具体要使用的数据集
                    type=str, help='VOC or COCO')
parser.add_argument('--dataset_root', default=VOC_ROOT,
                    help='Dataset root directory path')
parser.add_argument('--basenet', default='vgg16_reducedfc.pth',
                    help='Pretrained base model')
parser.add_argument('--batch_size', default=32, type=int,                   #batch_size=32, 即每跑一次iteration就处理32张图像样本
                    help='Batch size for training')
parser.add_argument('--resume', default=None, type=str,                     #用于指定预训练模型
                                                                             #此处的值为none表示不加载预训练模型，而是从头开始训练
                    help='Checkpoint state_dict file to resume training from')
parser.add_argument('--start_iter', default=0, type=int,
                    help='Resume training at this iter')
parser.add_argument('--num_workers', default=4, type=int,
                    help='Number of workers used in dataloading')
parser.add_argument('--cuda', default=True, type=str2bool,
                    help='Use CUDA to train model')
parser.add_argument('--lr', '--learning-rate', default=1e-3, type=float,    #lr，即学习率。影响模型的收敛速度
                    help='initial learning rate')
parser.add_argument('--momentum', default=0.9, type=float,
                    help='Momentum value for optim')
parser.add_argument('--weight_decay', default=5e-4, type=float,
                    help='Weight decay for SGD')
parser.add_argument('--gamma', default=0.1, type=float,
                    help='Gamma update for SGD')
parser.add_argument('--visdom', default=False, type=str2bool,
                    help='Use visdom for loss visualization')
parser.add_argument('--save_folder', default='weights/',                    #每到一个阶段，就在指定路径下保存当前时刻的模型参数
                    help='Directory for saving checkpoint models')
args = parser.parse_args(args=[]) 
#另外，还有部分超参数在'data\config.py'中设置

print(args)

Namespace(basenet='vgg16_reducedfc.pth', batch_size=32, cuda=True, dataset='VOC', dataset_root='/home/gong/wr/SSD/data/VOCdevkit/', gamma=0.1, lr=0.001, momentum=0.9, num_workers=4, resume=None, save_folder='weights/', start_iter=0, visdom=False, weight_decay=0.0005)


### 后续会用到的函数

In [6]:
def adjust_learning_rate(optimizer, gamma, step):
    lr = args.lr * (gamma ** (step))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


def xavier(param):
    init.xavier_uniform_(param)                                 


def weights_init(m):
    if isinstance(m, nn.Conv2d):
        xavier(m.weight.data)
        m.bias.data.zero_()


def create_vis_plot(_xlabel, _ylabel, _title, _legend):
    return viz.line(
        X=torch.zeros((1,)).cpu(),
        Y=torch.zeros((1, 3)).cpu(),
        opts=dict(
            xlabel=_xlabel,
            ylabel=_ylabel,
            title=_title,
            legend=_legend
        )
    )


def update_vis_plot(iteration, loc, conf, window1, window2, update_type,
                    epoch_size=1):
    viz.line(
        X=torch.ones((1, 3)).cpu() * iteration,
        Y=torch.Tensor([loc, conf, loc + conf]).unsqueeze(0).cpu() / epoch_size,
        win=window1,
        update=update_type
    )
    # initialize epoch plot on first iteration
    if iteration == 0:
        viz.line(
            X=torch.zeros((1, 3)).cpu(),
            Y=torch.Tensor([loc, conf, loc + conf]).unsqueeze(0).cpu(),
            win=window2,
            update=True
        )

### 数据集加载

In [7]:
if args.dataset == 'COCO':
    if args.dataset_root == VOC_ROOT:
        if not os.path.exists(COCO_ROOT):
            parser.error('Must specify dataset_root if specifying dataset')
        print("WARNING: Using default COCO dataset_root because " +
              "--dataset_root was not specified.")
        args.dataset_root = COCO_ROOT
    cfg = coco
    dataset = COCODetection(root=args.dataset_root,
                            transform=SSDAugmentation(cfg['min_dim'],
                                                      MEANS))
elif args.dataset == 'VOC':
    if args.dataset_root == COCO_ROOT:
        parser.error('Must specify dataset if specifying dataset_root')
    cfg = voc
    dataset = VOCDetection(root=args.dataset_root,
                           transform=SSDAugmentation(cfg['min_dim'],
                                                     MEANS))

### 判定使用GPU

In [8]:
if torch.cuda.is_available():
    if args.cuda:
        torch.set_default_tensor_type('torch.cuda.FloatTensor')
    if not args.cuda:
        print("WARNING: It looks like you have a CUDA device, but aren't " +
              "using CUDA.\nRun with --cuda for optimal training speed.")
        torch.set_default_tensor_type('torch.FloatTensor')
else:
    torch.set_default_tensor_type('torch.FloatTensor')

os.environ["CUDA_VISIBLE_DEVICES"] = "0"                                                 #只使用一块GPU来跑模型，“0”代表编号

if not os.path.exists(args.save_folder):      
    os.mkdir(args.save_folder)

### 创建一个SSD类的对象
SSD这个类中定义了网络的base部分（修改全连接层后的VGG16），extras部分（论文作者加入的多尺度feature map）和head部分（对选定的6个尺度下的feature map进行卷积操作得到的每个default box的每一个分类类别的confidence以及位置坐标的信息）

In [9]:
if args.visdom:
    import visdom
    viz = visdom.Visdom()

ssd_net = build_ssd('train', cfg['min_dim'], cfg['num_classes'])                     #返回SSD类的对象
net = ssd_net

if args.cuda:
    net = torch.nn.DataParallel(ssd_net)
    cudnn.benchmark = True

/home/gong/anaconda3/lib/python3.6/site-packages/torch/nn/parallel/data_parallel.py:26: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 1 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))


### 判断是否使用预训练模型

In [10]:
if args.resume:                                                                      #如果arg.resume的值不为none，则加载预训练模型
    print('Resuming training, loading {}...'.format(args.resume))
    ssd_net.load_weights(args.resume)
else:
    vgg_weights = torch.load(args.save_folder + args.basenet)
    print('Loading base network...')
    ssd_net.vgg.load_state_dict(vgg_weights)

if args.cuda:
    net = net.cuda()

if not args.resume:
    print('Initializing weights...')
    # initialize newly added layers' weights with xavier method
    ssd_net.extras.apply(weights_init)
    ssd_net.loc.apply(weights_init)
    ssd_net.conf.apply(weights_init)

Loading base network...
Initializing weights...


### 创建一个optimizer对象
每次迭代都会通过计算梯度来更新参数

In [11]:
optimizer = optim.SGD(net.parameters(), lr=args.lr, momentum=args.momentum,
                      weight_decay=args.weight_decay)

### 创建一个criterion对象
损失函数，计算预测和实际的差别

In [12]:
criterion = MultiBoxLoss(cfg['num_classes'], 0.5, True, 0, True, 3, 0.5,
                         False, args.cuda)

### 训练开始前的其他设置
同时也返回包括超参数设置在内的一些基本信息

In [13]:
net.train()
# loss counters
loc_loss = 0
conf_loss = 0
epoch = 0
print('Loading the dataset...')

epoch_size = len(dataset) // args.batch_size
print('Training SSD on:', dataset.name)
print('Using the specified args:')
print(args)

step_index = 0

if args.visdom:
    vis_title = 'SSD.PyTorch on ' + dataset.name
    vis_legend = ['Loc Loss', 'Conf Loss', 'Total Loss']
    iter_plot = create_vis_plot('Iteration', 'Loss', vis_title, vis_legend)
    epoch_plot = create_vis_plot('Epoch', 'Loss', vis_title, vis_legend)

Loading the dataset...
Training SSD on: VOC0712
Using the specified args:
Namespace(basenet='vgg16_reducedfc.pth', batch_size=32, cuda=True, dataset='VOC', dataset_root='/home/gong/wr/SSD/data/VOCdevkit/', gamma=0.1, lr=0.001, momentum=0.9, num_workers=4, resume=None, save_folder='weights/', start_iter=0, visdom=False, weight_decay=0.0005)


### 显示SSD的网络架构

In [14]:
net

DataParallel(
  (module): SSD(
    (vgg): ModuleList(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace)
      (16): MaxPool2d(kernel_size=2, stride=2,

### 开始训练！！！
### 对之前加载的数据集进行设置，并通过for循环开始迭代训练

In [ ]:

data_loader = data.DataLoader(dataset, args.batch_size,                    
                              num_workers=args.num_workers,              
                              shuffle=True, 
                              collate_fn=detection_collate,
                              pin_memory=True)
# create batch iterator
batch_iterator = iter(data_loader)
for iteration in range(args.start_iter, cfg['max_iter']): 

    #start_iter=0
    if args.visdom and iteration != 0 and (iteration % epoch_size == 0):        #epoch_size = len(dataset) // args.batch_size
        update_vis_plot(epoch, loc_loss, conf_loss, epoch_plot, None,
                        'append', epoch_size)
        # reset epoch loss counters
        loc_loss = 0
        conf_loss = 0
        epoch += 1           

    if iteration in cfg['lr_steps']:
        step_index += 1
        adjust_learning_rate(optimizer, args.gamma, step_index)

    try:                                                                         #每跑完一个epoch，就重新加载数据集
        images, targets = next(batch_iterator)
    except StopIteration:
        batch_iterator = iter(data_loader)
        images, targets = next(batch_iterator)

    if args.cuda:
        images = Variable(images.cuda())
        targets = [Variable(ann.cuda(), volatile=True) for ann in targets]
    else:
        images = Variable(images)
        targets = [Variable(ann, volatile=True) for ann in targets]   

    # forward
    t0 = time.time()
    out = net(images)
    # backprop
    optimizer.zero_grad()
    loss_l, loss_c = criterion(out, targets)
    loss = loss_l + loss_c
    loss.backward()
    optimizer.step()
    t1 = time.time()

    loc_loss += loss_l.item()
    conf_loss += loss_c.item()


    if iteration % 10 == 0:                                                         #每隔10个iteration，就记录一次并输出结果
        print('timer: %.4f sec.' % (t1 - t0))
        print('iter ' + repr(iteration) + ' || Loss: %.4f ||' % (loss.item()), end=' ')

    if args.visdom:
        update_vis_plot(iteration, loss_l.item(), loss_c.item(),
                        iter_plot, epoch_plot, 'append')

    if iteration != 0 and iteration % 5000 == 0:
        print('Saving state, iter:', iteration)
        torch.save(ssd_net.state_dict(), 'weights/ssd300_VOC_' +
                   repr(iteration) + '.pth')
torch.save(ssd_net.state_dict(),
           args.save_folder + '' + args.dataset + '.pth')

### 如下，为截取的部分训练过程

iter 119610 || Loss: 1.8245 || timer: 0.1810 sec.
iter 119620 || Loss: 1.7686 || timer: 0.1817 sec.
iter 119630 || Loss: 1.8669 || timer: 0.1802 sec.
iter 119640 || Loss: 2.1160 || timer: 0.1775 sec.
iter 119650 || Loss: 2.1764 || timer: 0.1922 sec.
iter 119660 || Loss: 1.5521 || timer: 0.1799 sec.
iter 119670 || Loss: 1.8507 || timer: 0.1803 sec.
iter 119680 || Loss: 1.9938 || timer: 0.1799 sec.
iter 119690 || Loss: 1.8818 || timer: 0.1991 sec.
iter 119700 || Loss: 1.5918 || timer: 0.1814 sec.
iter 119710 || Loss: 2.2675 || timer: 0.1802 sec.
iter 119720 || Loss: 1.5979 || timer: 0.1839 sec.
iter 119730 || Loss: 2.1664 || timer: 0.1797 sec.
iter 119740 || Loss: 1.8714 || timer: 0.1791 sec.
iter 119750 || Loss: 1.9588 || timer: 0.1793 sec.
iter 119760 || Loss: 2.2306 || timer: 0.1817 sec.
iter 119770 || Loss: 1.6182 || timer: 0.1907 sec.
iter 119780 || Loss: 1.8423 || timer: 0.2198 sec.
iter 119790 || Loss: 2.3274 || timer: 0.1801 sec.
iter 119800 || Loss: 1.8025 || timer: 0.1824 sec.
iter 119810 || Loss: 1.5610 || timer: 0.1804 sec.
iter 119820 || Loss: 1.8080 || timer: 0.1817 sec.
iter 119830 || Loss: 2.0728 || timer: 0.1819 sec.
iter 119840 || Loss: 1.8988 || timer: 0.1807 sec.
iter 119850 || Loss: 1.9533 || timer: 0.1825 sec.
iter 119860 || Loss: 2.0571 || timer: 0.1807 sec.
iter 119870 || Loss: 2.0174 || timer: 0.1812 sec.
iter 119880 || Loss: 1.7564 || timer: 0.1818 sec.
iter 119890 || Loss: 1.5111 || timer: 0.1848 sec.
iter 119900 || Loss: 2.0554 || timer: 0.1786 sec.
iter 119910 || Loss: 2.2887 || timer: 0.1781 sec.
iter 119920 || Loss: 2.0812 || timer: 0.1849 sec.
iter 119930 || Loss: 2.3477 || timer: 0.1812 sec.
iter 119940 || Loss: 1.6422 || timer: 0.1792 sec.
iter 119950 || Loss: 1.7281 || timer: 0.1817 sec.
iter 119960 || Loss: 2.1319 || timer: 0.1800 sec.
iter 119970 || Loss: 1.6839 || timer: 0.1802 sec.
iter 119980 || Loss: 1.7383 || timer: 0.1796 sec.

### 在测试集上进行测试

In [16]:
run eval.py

Finished loading model!
im_detect: 1/4952 0.825s
im_detect: 2/4952 0.040s
im_detect: 3/4952 0.035s
im_detect: 4/4952 0.028s
im_detect: 5/4952 0.022s
im_detect: 6/4952 0.024s
im_detect: 7/4952 0.020s
im_detect: 8/4952 0.012s
im_detect: 9/4952 0.011s
im_detect: 10/4952 0.066s
im_detect: 11/4952 0.019s
im_detect: 12/4952 0.010s
im_detect: 13/4952 0.014s
im_detect: 14/4952 0.030s
im_detect: 15/4952 0.011s
im_detect: 16/4952 0.011s
im_detect: 17/4952 0.010s
im_detect: 18/4952 0.019s
im_detect: 19/4952 0.010s
im_detect: 20/4952 0.015s
im_detect: 21/4952 0.016s
im_detect: 22/4952 0.017s
im_detect: 23/4952 0.010s
im_detect: 24/4952 0.010s
im_detect: 25/4952 0.020s
im_detect: 26/4952 0.032s
im_detect: 27/4952 0.060s
im_detect: 28/4952 0.016s
im_detect: 29/4952 0.015s
im_detect: 30/4952 0.040s
im_detect: 31/4952 0.040s
im_detect: 32/4952 0.013s
im_detect: 33/4952 0.011s
im_detect: 34/4952 0.012s
im_detect: 35/4952 0.043s
im_detect: 36/4952 0.066s
im_detect: 37/4952 0.017s
im_detect: 38/4952 0.01

im_detect: 313/4952 0.042s
im_detect: 314/4952 0.033s
im_detect: 315/4952 0.014s
im_detect: 316/4952 0.043s
im_detect: 317/4952 0.027s
im_detect: 318/4952 0.047s
im_detect: 319/4952 0.019s
im_detect: 320/4952 0.031s
im_detect: 321/4952 0.016s
im_detect: 322/4952 0.016s
im_detect: 323/4952 0.021s
im_detect: 324/4952 0.022s
im_detect: 325/4952 0.035s
im_detect: 326/4952 0.046s
im_detect: 327/4952 0.014s
im_detect: 328/4952 0.012s
im_detect: 329/4952 0.022s
im_detect: 330/4952 0.039s
im_detect: 331/4952 0.026s
im_detect: 332/4952 0.023s
im_detect: 333/4952 0.022s
im_detect: 334/4952 0.019s
im_detect: 335/4952 0.023s
im_detect: 336/4952 0.029s
im_detect: 337/4952 0.016s
im_detect: 338/4952 0.040s
im_detect: 339/4952 0.023s
im_detect: 340/4952 0.041s
im_detect: 341/4952 0.011s
im_detect: 342/4952 0.019s
im_detect: 343/4952 0.013s
im_detect: 344/4952 0.012s
im_detect: 345/4952 0.033s
im_detect: 346/4952 0.011s
im_detect: 347/4952 0.024s
im_detect: 348/4952 0.031s
im_detect: 349/4952 0.013s
i

im_detect: 617/4952 0.039s
im_detect: 618/4952 0.051s
im_detect: 619/4952 0.012s
im_detect: 620/4952 0.037s
im_detect: 621/4952 0.022s
im_detect: 622/4952 0.014s
im_detect: 623/4952 0.028s
im_detect: 624/4952 0.052s
im_detect: 625/4952 0.038s
im_detect: 626/4952 0.031s
im_detect: 627/4952 0.033s
im_detect: 628/4952 0.048s
im_detect: 629/4952 0.028s
im_detect: 630/4952 0.035s
im_detect: 631/4952 0.032s
im_detect: 632/4952 0.047s
im_detect: 633/4952 0.011s
im_detect: 634/4952 0.030s
im_detect: 635/4952 0.011s
im_detect: 636/4952 0.028s
im_detect: 637/4952 0.059s
im_detect: 638/4952 0.031s
im_detect: 639/4952 0.069s
im_detect: 640/4952 0.040s
im_detect: 641/4952 0.014s
im_detect: 642/4952 0.032s
im_detect: 643/4952 0.016s
im_detect: 644/4952 0.013s
im_detect: 645/4952 0.018s
im_detect: 646/4952 0.018s
im_detect: 647/4952 0.012s
im_detect: 648/4952 0.016s
im_detect: 649/4952 0.018s
im_detect: 650/4952 0.012s
im_detect: 651/4952 0.028s
im_detect: 652/4952 0.044s
im_detect: 653/4952 0.032s
i

im_detect: 921/4952 0.036s
im_detect: 922/4952 0.015s
im_detect: 923/4952 0.045s
im_detect: 924/4952 0.044s
im_detect: 925/4952 0.031s
im_detect: 926/4952 0.034s
im_detect: 927/4952 0.030s
im_detect: 928/4952 0.013s
im_detect: 929/4952 0.042s
im_detect: 930/4952 0.017s
im_detect: 931/4952 0.015s
im_detect: 932/4952 0.015s
im_detect: 933/4952 0.022s
im_detect: 934/4952 0.018s
im_detect: 935/4952 0.021s
im_detect: 936/4952 0.058s
im_detect: 937/4952 0.028s
im_detect: 938/4952 0.014s
im_detect: 939/4952 0.015s
im_detect: 940/4952 0.012s
im_detect: 941/4952 0.021s
im_detect: 942/4952 0.032s
im_detect: 943/4952 0.024s
im_detect: 944/4952 0.028s
im_detect: 945/4952 0.025s
im_detect: 946/4952 0.015s
im_detect: 947/4952 0.015s
im_detect: 948/4952 0.018s
im_detect: 949/4952 0.016s
im_detect: 950/4952 0.022s
im_detect: 951/4952 0.024s
im_detect: 952/4952 0.067s
im_detect: 953/4952 0.031s
im_detect: 954/4952 0.017s
im_detect: 955/4952 0.039s
im_detect: 956/4952 0.012s
im_detect: 957/4952 0.021s
i

im_detect: 1221/4952 0.014s
im_detect: 1222/4952 0.016s
im_detect: 1223/4952 0.033s
im_detect: 1224/4952 0.013s
im_detect: 1225/4952 0.042s
im_detect: 1226/4952 0.044s
im_detect: 1227/4952 0.012s
im_detect: 1228/4952 0.041s
im_detect: 1229/4952 0.028s
im_detect: 1230/4952 0.034s
im_detect: 1231/4952 0.015s
im_detect: 1232/4952 0.021s
im_detect: 1233/4952 0.027s
im_detect: 1234/4952 0.015s
im_detect: 1235/4952 0.025s
im_detect: 1236/4952 0.040s
im_detect: 1237/4952 0.058s
im_detect: 1238/4952 0.034s
im_detect: 1239/4952 0.014s
im_detect: 1240/4952 0.031s
im_detect: 1241/4952 0.032s
im_detect: 1242/4952 0.029s
im_detect: 1243/4952 0.023s
im_detect: 1244/4952 0.017s
im_detect: 1245/4952 0.043s
im_detect: 1246/4952 0.045s
im_detect: 1247/4952 0.066s
im_detect: 1248/4952 0.021s
im_detect: 1249/4952 0.020s
im_detect: 1250/4952 0.011s
im_detect: 1251/4952 0.049s
im_detect: 1252/4952 0.020s
im_detect: 1253/4952 0.066s
im_detect: 1254/4952 0.027s
im_detect: 1255/4952 0.026s
im_detect: 1256/4952

...

im_detect: 4178/4952 0.038s
im_detect: 4179/4952 0.023s
im_detect: 4180/4952 0.104s
im_detect: 4181/4952 0.023s
im_detect: 4182/4952 0.012s
im_detect: 4183/4952 0.018s
im_detect: 4184/4952 0.040s
im_detect: 4185/4952 0.026s
im_detect: 4186/4952 0.030s
im_detect: 4187/4952 0.013s
im_detect: 4188/4952 0.014s
im_detect: 4189/4952 0.011s
im_detect: 4190/4952 0.027s
im_detect: 4191/4952 0.033s
im_detect: 4192/4952 0.033s
im_detect: 4193/4952 0.037s
im_detect: 4194/4952 0.036s
im_detect: 4195/4952 0.015s
im_detect: 4196/4952 0.030s
im_detect: 4197/4952 0.015s
im_detect: 4198/4952 0.029s
im_detect: 4199/4952 0.030s
im_detect: 4200/4952 0.044s
im_detect: 4201/4952 0.038s
im_detect: 4202/4952 0.025s
im_detect: 4203/4952 0.026s
im_detect: 4204/4952 0.040s
im_detect: 4205/4952 0.064s
im_detect: 4206/4952 0.018s
im_detect: 4207/4952 0.026s
im_detect: 4208/4952 0.042s
im_detect: 4209/4952 0.019s
im_detect: 4210/4952 0.055s
im_detect: 4211/4952 0.016s
im_detect: 4212/4952 0.027s
im_detect: 4213/4952

im_detect: 4472/4952 0.012s
im_detect: 4473/4952 0.011s
im_detect: 4474/4952 0.013s
im_detect: 4475/4952 0.021s
im_detect: 4476/4952 0.017s
im_detect: 4477/4952 0.022s
im_detect: 4478/4952 0.041s
im_detect: 4479/4952 0.026s
im_detect: 4480/4952 0.015s
im_detect: 4481/4952 0.020s
im_detect: 4482/4952 0.016s
im_detect: 4483/4952 0.060s
im_detect: 4484/4952 0.025s
im_detect: 4485/4952 0.014s
im_detect: 4486/4952 0.053s
im_detect: 4487/4952 0.020s
im_detect: 4488/4952 0.038s
im_detect: 4489/4952 0.026s
im_detect: 4490/4952 0.026s
im_detect: 4491/4952 0.058s
im_detect: 4492/4952 0.021s
im_detect: 4493/4952 0.030s
im_detect: 4494/4952 0.040s
im_detect: 4495/4952 0.050s
im_detect: 4496/4952 0.013s
im_detect: 4497/4952 0.059s
im_detect: 4498/4952 0.017s
im_detect: 4499/4952 0.036s
im_detect: 4500/4952 0.028s
im_detect: 4501/4952 0.035s
im_detect: 4502/4952 0.027s
im_detect: 4503/4952 0.034s
im_detect: 4504/4952 0.027s
im_detect: 4505/4952 0.016s
im_detect: 4506/4952 0.038s
im_detect: 4507/4952

im_detect: 4770/4952 0.035s
im_detect: 4771/4952 0.028s
im_detect: 4772/4952 0.022s
im_detect: 4773/4952 0.040s
im_detect: 4774/4952 0.029s
im_detect: 4775/4952 0.012s
im_detect: 4776/4952 0.039s
im_detect: 4777/4952 0.022s
im_detect: 4778/4952 0.013s
im_detect: 4779/4952 0.013s
im_detect: 4780/4952 0.018s
im_detect: 4781/4952 0.012s
im_detect: 4782/4952 0.042s
im_detect: 4783/4952 0.010s
im_detect: 4784/4952 0.018s
im_detect: 4785/4952 0.029s
im_detect: 4786/4952 0.033s
im_detect: 4787/4952 0.048s
im_detect: 4788/4952 0.060s
im_detect: 4789/4952 0.030s
im_detect: 4790/4952 0.018s
im_detect: 4791/4952 0.014s
im_detect: 4792/4952 0.018s
im_detect: 4793/4952 0.011s
im_detect: 4794/4952 0.020s
im_detect: 4795/4952 0.096s
im_detect: 4796/4952 0.022s
im_detect: 4797/4952 0.013s
im_detect: 4798/4952 0.046s
im_detect: 4799/4952 0.015s
im_detect: 4800/4952 0.011s
im_detect: 4801/4952 0.024s
im_detect: 4802/4952 0.018s
im_detect: 4803/4952 0.053s
im_detect: 4804/4952 0.024s
im_detect: 4805/4952

/home/gong/wr/SSD/eval.py:153: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  if dets == []:


Writing boat VOC results file
Writing bottle VOC results file
Writing bus VOC results file
Writing car VOC results file
Writing cat VOC results file
Writing chair VOC results file
Writing cow VOC results file
Writing diningtable VOC results file
Writing dog VOC results file
Writing horse VOC results file
Writing motorbike VOC results file
Writing person VOC results file
Writing pottedplant VOC results file
Writing sheep VOC results file
Writing sofa VOC results file
Writing train VOC results file
Writing tvmonitor VOC results file
VOC07 metric? Yes
AP for aeroplane = 0.8207
AP for bicycle = 0.8568
AP for bird = 0.7546
AP for boat = 0.6952
AP for bottle = 0.5019
AP for bus = 0.8479
AP for car = 0.8584
AP for cat = 0.8734
AP for chair = 0.6136
AP for cow = 0.8243
AP for diningtable = 0.7906
AP for dog = 0.8566
AP for horse = 0.8714
AP for motorbike = 0.8403
AP for person = 0.7895
AP for pottedplant = 0.5069
AP for sheep = 0.7767
AP for sofa = 0.7894
AP for train = 0.8623
AP for tvmonitor